# Import required libraries

In [1]:
import pandas as pd
import os
import librosa
import re
import numpy as np
import tensorflow as tf

C:\Users\rishi\AppData\Local\Temp\ipykernel_28836\3579895535.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from tensorflow.keras import layers, models

In [3]:
from sklearn.model_selection import train_test_split 

# Pre process data

In [4]:
#import the csv with recording locations, ID, outcome
#C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data
df = pd.read_csv(r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data\Data\training_data.csv")

In [5]:
df.head(5)

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.9,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.9,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN


In [6]:
df["Outcome"].value_counts()

Outcome
Normal      486
Abnormal    456
Name: count, dtype: int64

In [7]:
def pad_array(data, signal_length = None):
    max_len = 0
    for i in data:
        if len(i) > max_len:
            max_len = len(i)
    if not signal_length == None:
        max_len = signal_length
    new_arr = np.zeros((len(data),max_len))
    for j in range(len(data)):
        if len(data[j]) > max_len:
            new_arr[j,:] = data[j][:max_len]
        else:
            new_arr[j,:len(data[j])] = data[j]
    return new_arr

In [8]:
def pad_signal(data, max_len = int(1*355712)):
    #pad_length - 1422490 = max_length of the signals in the dataset
    #pad_length after spectrogram = 355712 = max length of the spectrograms
    new_arr = np.zeros(max_len)
    if len(data) > max_len:
        new_arr = data[:max_len]
    else:
        new_arr[:len(data)] = data
    return new_arr

### flat Specs

In [36]:
#get all files that starts with the ID and ends with .wav
#make a list of lists for each ID
outcome_classes = ["Normal", "Abnormal"]
specs = []
#wav_files = []
outcomes = []
for i in range(df.shape[0]):
    patient_ID = df["Patient ID"][i]
    outcome = df["Outcome"][i]
    specs_for_each_patient = []
    #wav_file_for_each_patient = []
    number_of_recordings = 0
    data_directory = "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/circor dataset/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data"


    for root, dirs, files in os.walk(data_directory):
        for file in files:

            if re.search("^" + str(patient_ID) + "[A-Z_]{3}.wav$", file):
                number_of_recordings += 1
                #load wav file
                audio_signal, sampling_rate = librosa.load(data_directory + '/'+ file)
                
                #get the spectrogram
                spec = librosa.feature.melspectrogram(y=audio_signal, sr=sampling_rate)
                spec = spec.flatten()

                #pad spectrogram - pad after taking spectrogram because 
                #the length of spectrogram is different for each audio signal(even if len of audio signal is same)
                #we need data with same length for training
                padded_spec = pad_signal(spec)

                specs_for_each_patient.append(padded_spec)
                #wav_file_for_each_patient.append(audio_signal)

    #add outcome for each patient
    current_outcome = np.zeros(2, dtype=int)
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1

    for i in range(number_of_recordings):
        outcomes.append(current_outcome) #add the outcome for each location of a patient

    #wav_files += wav_file_for_each_patient
    specs += specs_for_each_patient


## Not flat specs

In [15]:
#get all files that starts with the ID and ends with .wav
#make a list of lists for each ID
outcome_classes = ["Normal", "Abnormal"]
specs = []
#wav_files = []
outcomes = []
for i in range(df.shape[0]):
    patient_ID = df["Patient ID"][i]
    outcome = df["Outcome"][i]
    specs_for_each_patient = []
    #wav_file_for_each_patient = []
    number_of_recordings = 0
    data_directory = "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/circor dataset/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data"


    for root, dirs, files in os.walk(data_directory):
        for file in files:

            if re.search("^" + str(patient_ID) + "[A-Z_]{3}.wav$", file):
                number_of_recordings += 1
                #load wav file
                audio_signal, sampling_rate = librosa.load(data_directory + '/'+ file)
                padded_spec = pad_signal(audio_signal, max_len=1422490)

                #get the spectrogram
                spec = librosa.feature.melspectrogram(y=padded_spec, sr=sampling_rate)
                #spec = spec.flatten()

                #pad spectrogram - pad after taking spectrogram because 
                #the length of spectrogram is different for each audio signal(even if len of audio signal is same)
                #we need data with same length for training
                #

                specs_for_each_patient.append(spec)
                #wav_file_for_each_patient.append(audio_signal)

    #add outcome for each patient
    current_outcome = np.zeros(2, dtype=int)
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1

    for i in range(number_of_recordings):
        outcomes.append(current_outcome) #add the outcome for each location of a patient

    #wav_files += wav_file_for_each_patient
    specs += specs_for_each_patient


In [1]:
specs = np.load(r"Preprocessed data/specs.npy")


In [5]:
outcomes = np.load(r"Preprocessed data/outcomes_array.npy")

## Short not flat spectograms

In [21]:
#get all files that starts with the ID and ends with .wav
#make a list of lists for each ID
outcome_classes = ["Normal", "Abnormal"]
specs = []
#wav_files = []
outcomes = []
for i in range(df.shape[0]): #
    patient_ID = df["Patient ID"][i]
    outcome = df["Outcome"][i]
    specs_for_each_patient = []
    #wav_file_for_each_patient = []
    number_of_recordings = 0
    data_directory = "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/circor dataset/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data"


    for root, dirs, files in os.walk(data_directory):
        for file in files:

            if re.search("^" + str(patient_ID) + "[A-Z_]{3}.wav$", file):
                number_of_recordings += 1
                #load wav file
                audio_signal, sampling_rate = librosa.load(data_directory + '/'+ file)
                
                #length of the file = time in sec * sample_rate
                #sample rate = samples(data points) per second
                #lets take 10 sec sample
                #max_len = 10 * sampling_rate
                padded_audio = pad_signal(audio_signal, max_len=10*sampling_rate)

                #get the spectrogram
                spec = librosa.feature.melspectrogram(y=padded_audio, sr=sampling_rate)
                #spec = spec.flatten()

                #pad spectrogram - pad after taking spectrogram because 
                #the length of spectrogram is different for each audio signal(even if len of audio signal is same)
                #we need data with same length for training
                #

                specs_for_each_patient.append(spec)
                #wav_file_for_each_patient.append(audio_signal)

    #add outcome for each patient
    current_outcome = np.zeros(2, dtype=int)
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1

    for i in range(number_of_recordings):
        outcomes.append(current_outcome) #add the outcome for each location of a patient

    #wav_files += wav_file_for_each_patient
    specs += specs_for_each_patient


In [22]:
len(specs)

3118

In [23]:
specs[1].shape

(128, 431)

In [25]:
specs = np.array(specs)

In [26]:
#number of channels in each sample is 1 in grayscale image
#CNN requires 4D input (batch_size, height, width, channels)
specs = specs.reshape(specs.shape[0], specs.shape[1], specs.shape[2], 1)

In [27]:
#splitting the data into training and testing
specs_train, specs_test, outcomes_train, outcomes_test = train_test_split(specs, outcomes, test_size=0.2, random_state=42)

In [28]:
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/Preprocessed data/specs_train_s.npy",
        specs_train)
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/Preprocessed data/specs_test_S.npy",
        specs_test)
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/Preprocessed data/outcomes_train.npy",
        outcomes_train)
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/Preprocessed data/outcomes_test.npy",
        outcomes_test)

In [16]:
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/specs.npy",
        specs)

In [41]:
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/outcomes_array.npy",
        np.array(outcomes))

## Window segment split 5 sec

In [11]:
import random
#get indices for training and testing set before segmentation
#so that a patients info is not present in both training and testing set
train_index = random.sample(range(df.shape[0]), int(df.shape[0]*0.8))
test_index = list(set(range(df.shape[0])) - set(train_index))

In [22]:
#--------------------------------------preprocessing train data
#get all files that starts with the ID and ends with .wav
#make a list of lists for each ID
outcome_classes = ["Normal", "Abnormal"]
specs = []
#wav_files = []
outcomes = []
for i in train_index: #
    patient_ID = df["Patient ID"][i]
    outcome = df["Outcome"][i]
    specs_for_each_patient = []
    #wav_file_for_each_patient = []
    number_of_recordings = 0
    data_directory = r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data\Data\training_data"


    for root, dirs, files in os.walk(data_directory):
        for file in files:

            if re.search("^" + str(patient_ID) + "[A-Z_]{3}.wav$", file):
                #for loop - range = audio_len/5. 
                
                #load wav file
                audio_signal, sampling_rate = librosa.load(data_directory + '/'+ file)

                for i in range(0, len(audio_signal), 5*sampling_rate):
                    audio_signal_chunk = audio_signal[i:i+5*sampling_rate]
                    #you dont need the step below ????
                    padded_audio = pad_signal(audio_signal_chunk, max_len=5*sampling_rate)

                    #get the spectrogram
                    spec = librosa.feature.melspectrogram(y=padded_audio, sr=sampling_rate)

                    specs_for_each_patient.append(spec)
                    number_of_recordings += 1

                

    #add outcome for each patient
    current_outcome = np.zeros(2, dtype=int)
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1

    for i in range(number_of_recordings):
        outcomes.append(current_outcome) #add the outcome for each location of a patient

    #wav_files += wav_file_for_each_patient
    specs += specs_for_each_patient

train_physionet_specs = np.array(specs)
train_physionet_outcome = outcomes


In [23]:
#---------------------preprocessing for test data
#get all files that starts with the ID and ends with .wav
#make a list of lists for each ID
outcome_classes = ["Normal", "Abnormal"]
specs = []
#wav_files = []
outcomes = []
for i in test_index: #
    patient_ID = df["Patient ID"][i]
    outcome = df["Outcome"][i]
    specs_for_each_patient = []
    #wav_file_for_each_patient = []
    number_of_recordings = 0
    data_directory = r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data\Data\training_data"


    for root, dirs, files in os.walk(data_directory):
        for file in files:

            if re.search("^" + str(patient_ID) + "[A-Z_]{3}.wav$", file):
                #for loop - range = audio_len/5. 
                
                #load wav file
                audio_signal, sampling_rate = librosa.load(data_directory + '/'+ file)

                for i in range(0, len(audio_signal), 5*sampling_rate):
                    audio_signal_chunk = audio_signal[i:i+5*sampling_rate]
                    #you dont need the step below ????
                    padded_audio = pad_signal(audio_signal_chunk, max_len=5*sampling_rate)

                    #get the spectrogram
                    spec = librosa.feature.melspectrogram(y=padded_audio, sr=sampling_rate)

                    specs_for_each_patient.append(spec)
                    number_of_recordings += 1

                

    #add outcome for each patient
    current_outcome = np.zeros(2, dtype=int)
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1

    for i in range(number_of_recordings):
        outcomes.append(current_outcome) #add the outcome for each location of a patient

    #wav_files += wav_file_for_each_patient
    specs += specs_for_each_patient

test_physionet_specs = np.array(specs)
test_physionet_outcome = outcomes


In [27]:
print(train_physionet_specs.shape, test_physionet_specs.shape)
print(len(train_physionet_outcome), len(test_physionet_outcome))

(12560, 128, 216) (3290, 128, 216)
12560 3290


In [28]:
np.save( r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data\Preprocessed data\Segmented\train_physionet_specs_segmented.npy",
        train_physionet_specs)
np.save( r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data\Preprocessed data\Segmented\train_physionet_outcome_segmented.npy",
        train_physionet_outcome)
np.save( r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data\Preprocessed data\Segmented\test_physionet_specs_segmented.npy",
        test_physionet_specs)
np.save( r"C:\Users\rishi\OneDrive - Monash University\Documents\Monash\Sonorus\sonorus - physionet data\Preprocessed data\Segmented\test_physionet_outcome_segmented.npy",
        test_physionet_outcome)

In [31]:
#dont need this
#splitting the data into training and testing
specs_train, specs_test, outcomes_train, outcomes_test = train_test_split(specs, outcomes, test_size=0.2, random_state=42)

np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/sonorus - physionet data/Preprocessed data/specs_train_segmented.npy",
        specs_train)
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/sonorus - physionet data/Preprocessed data/specs_test_segmented.npy",
        specs_test)
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/sonorus - physionet data/Preprocessed data/outcomes_train_segmented.npy",
        outcomes_train)
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/sonorus - physionet data/Preprocessed data/outcomes_test_segmented.npy",
        outcomes_test)

### Wav files

In [7]:
#get all files that starts with the ID and ends with .wav
#make a list of lists for each ID
outcome_classes = ["Normal", "Abnormal"]
#specs = []
wav_files = []
outcomes = []
for i in range(df.shape[0]):
    patient_ID = df["Patient ID"][i]
    outcome = df["Outcome"][i]
    specs_for_each_patient = []
    wav_file_for_each_patient = []
    number_of_recordings = 0
    data_directory = "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/circor dataset/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data"


    for root, dirs, files in os.walk(data_directory):
        for file in files:

            if re.search("^" + str(patient_ID) + "[A-Z_]{3}.wav$", file):
                number_of_recordings += 1
                #load wav file
                audio_signal, sampling_rate = librosa.load(data_directory + '/'+ file)
                
                #get the spectrogram
                #spec = librosa.feature.melspectrogram(y=audio_signal, sr=sampling_rate)
                #spec = spec.flatten()

                #pad spectrogram - pad after taking spectrogram because 
                #the length of spectrogram is different for each audio signal(even if len of audio signal is same)
                #we need data with same length for training
                padded_audio = pad_signal(audio_signal, max_len=1422490)

                #specs_for_each_patient.append(padded_spec)
                wav_file_for_each_patient.append(padded_audio)

    #add outcome for each patient
    current_outcome = np.zeros(2, dtype=int)
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1

    for i in range(number_of_recordings):
        outcomes.append(current_outcome) #add the outcome for each location of a patient

    wav_files += wav_file_for_each_patient
    #specs += specs_for_each_patient


In [8]:
wav_files = np.array(wav_files)

MemoryError: Unable to allocate 33.0 GiB for an array with shape (3118, 1422490) and data type float64

In [9]:
np.save( "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/wav_files.npy",
        wav_files)

MemoryError: Unable to allocate 33.0 GiB for an array with shape (3118, 1422490) and data type float64

## Old

In [ ]:
#get all files that starts with the ID and ends with .wav
#make a list of lists for each ID
outcome_classes = ["Normal", "Abnormal"]
specs = np.array([])
#wav_files = []
outcomes = []
for i in range(df.shape[0]):
    patient_ID = df["Patient ID"][i]
    outcome = df["Outcome"][i]
    specs_for_each_patient = np.array([])
    #wav_file_for_each_patient = []
    number_of_recordings = 0
    data_directory = "C:/Users/rishi/OneDrive - Monash University/Documents/Monash/sonorus/circor dataset/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data"


    for root, dirs, files in os.walk(data_directory):
        for file in files:

            if re.search("^" + str(patient_ID) + "[A-Z_]{3}.wav$", file):
                number_of_recordings += 1
                #load wav file
                audio_signal, sampling_rate = librosa.load(data_directory + '/'+ file)
                
                #get the spectrogram
                spec = librosa.feature.melspectrogram(y=audio_signal, sr=sampling_rate)
                spec = spec.flatten()

                #pad spectrogram - pad after taking spectrogram because 
                #the length of spectrogram is different for each audio signal(even if len of audio signal is same)
                #we need data with same length for training
                padded_spec = pad_signal(spec)

                specs_for_each_patient = np.append(specs_for_each_patient, padded_spec)
                #wav_file_for_each_patient.append(audio_signal)

    #add outcome for each patient
    current_outcome = np.zeros(2, dtype=int)
    j = outcome_classes.index(outcome)
    current_outcome[j] = 1

    for i in range(number_of_recordings):
        outcomes.append(current_outcome) #add the outcome for each location of a patient

    #wav_files += wav_file_for_each_patient
    specs = np.append(specs, specs_for_each_patient)
